In [4]:
# datasets for BERT
!pip install datasets
from datasets import load_dataset
import pandas as pd
# relevant packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
torch.cuda.empty_cache()
from tqdm.notebook import tqdm
# random check 
print(torch.__version__)

df = pd.read_csv("tweet_emotions.csv")

# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy

# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en_core_web_sm

!pip install -U symspellpy

#for spell and slang correction
!pip install gingerit
from gingerit.gingerit import GingerIt

#for emoticons
!pip install emot --upgrade
import emot 
emot_obj = emot.core.emot() 

from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import re, string, json
import spacy
from datasets import Value, ClassLabel, Features, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

1.13.1+cu116



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 12.8/12.8 MB 38.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#we want to remove small datasets 
# df = df[~df.sentiment.str.contains('\|')]   
df = df[df.sentiment != 'anger'] #& 'boredom' & 'enthusiasm' & 'empty'
df = df[df.sentiment != 'boredom']
df = df[df.sentiment != 'enthusiasm']
df = df[df.sentiment != 'empty']
df = df[df.sentiment != 'sentiment'] #there is sentiment in sentiments!
df = df[df.sentiment != 'fun']
df = df[df.sentiment != 'relief']
df = df[df.sentiment != 'hate']
df = df[df.sentiment != 'surprise']

df.sentiment.value_counts()
#class imbalance

neutral      8638
worry        8459
happiness    5209
sadness      5165
love         3842
Name: sentiment, dtype: int64

In [7]:
#build dictionary, key: emotion, value: 
possible_labels = df.sentiment.unique()
label_dict = {}
#loop over index
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)

{'sadness': 0, 'neutral': 1, 'worry': 2, 'love': 3, 'happiness': 4}


In [8]:
from datasets import Value, ClassLabel, Features, Dataset

#build new column for these values
df["label"] = df.sentiment.replace(label_dict)

features = Features({"label": Value("int64"), "label": ClassLabel(num_classes=5, names=[0,1,2,3,4])})

In [9]:
from sklearn.model_selection import train_test_split
#stratified split
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                 df.label.values,
                                                 test_size = 0.025,
                                                 stratify = df.label.values
                                                 )

df['data_type'] = ['not_set']*df.shape[0]

df.head()

,tweet_id,sentiment,content,label,data_type
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,0,not_set
2,1956967696,sadness,Funeral ceremony...gloomy friday...,0,not_set
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,1,not_set
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...,2,not_set
6,1956968487,sadness,"I should be sleep, but im not! thinking about ...",0,not_set


In [10]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['sentiment', 'label', 'data_type']).count()
#group by using count

tweet_id  content
sentiment label data_type                   
happiness 4     train          5079     5079
                val             130      130
love      3     train          3746     3746
                val              96       96
neutral   1     train          8422     8422
                val             216      216
sadness   0     train          5036     5036
                val             129      129
worry     2     train          8247     8247
                val             212      212

In [11]:
def normalization_pipeline(sentences):
    print("##############################")
    print("Starting Normalization Process")
    sentences = _simplify_punctuation_and_whitespace(sentences) # !!!!! "      "
    sentences = _normalize_contractions(sentences) #also corrects spelling now
    print("Normalization Process Finished")
    print("##############################")
    return sentences

    
def _simplify_punctuation_and_whitespace(sentence_list):
    """
    words with more than 4 all-capital words will get <-EMPW 
    """
    norm_sents = []
    print("Replacing -URL- , Replacing @MENTION and #HASHTAG, Reducing character repetitions, ")
    print("Simplifying punctuation, Removing whitespaces")

    for sentence in tqdm(sentence_list):
        sent = _replace_urls(sentence)
        sent = _mention_hash(sent)
        sent = _simplify_punctuation(sent)
        sent = _reduce_repetitions(sent)
        sent = _normalize_whitespace(sent)
        norm_sents.append(sent)
    return norm_sents


def _replace_urls(text):
    url_regex = r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
    text = re.sub(url_regex, "-URL-", text)
    return text


def _mention_hash(in_str):
    """
     @MENTIONs and #HASHTAGs will take forms of @men and #has 
    note: BEWARE OF USES OF # AND @ AND SPACES BETWEEN THEM
    """
    in_str = str(in_str)
    in_str = re.sub('@\w+', '@MEN', in_str,flags=re.IGNORECASE) # use @\w+ for word replacement or @ with space after @MEN for keeping mention
    in_str = re.sub('#', '#HAS ', in_str,flags=re.IGNORECASE)
    in_str = re.sub(r'([\w])\1+', r'\1\1', in_str) #reduce repeated characters to 2
    return in_str.strip()


def _simplify_punctuation(text):
    """
    puntuations like '!!!!!' will be transformed into '!! <-EMPP'
    This function simplifies doubled or more complex punctuation. The exception is '...'. #?! ??? !!!
    """
    corrected = str(text)
    corrected = re.sub(r'([!?,;])\1+', r'\1\1 <-EMPP', corrected) #\1\1 makes it to 2 consecutive punctuation
    corrected = re.sub(r'\.{2,}', r'...', corrected)
    return corrected

def _reduce_repetitions(text):
    """
    Auxiliary function to help with exxagerated (repeated characters in) words.
    Examples:
        woooooords -> woords <-EMPW
        dooorwaaay -> doorwaay <-EMPW
        SICK -> sick <-EMPU
    """
    correction = str(text)
    for index, words in enumerate(str(text).split()):
        if _is_EMP_word(words)==True :
            #insert EMPW after word
            correction = correction.replace(words, words + ' <-EMPW')
        if (len(words) > 4) & (words.isupper()==True) & (words[0] not in string.punctuation):
            correction = correction.replace(words, words + ' <-EMPU')
    #TODO work on complexity reduction.
    return re.sub(r'([\w])\1+', r'\1\1', correction) #\1\1 will only keep 2 consecutive characters


def _is_EMP_word(word):
    """
    True/ False: checks if the word has 3 consecutive characters"""
    count=1
    if len(word)>1:
        for i in range(1,len(word)):
            if word[i] in string.punctuation: #this function is only for words!
                return False
            if word[i-1]==word[i]:
                count+=1
                if(count>=3):
                     return True
            else :
                if(count>=3):
                    return True
                count=1
    else :
        return False
    return False


def _normalize_whitespace(text):
    """
    normalizes whitespaces, removing duplicates.
    """
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")
    
    
#Substitution of contractions:  -----------------------------------------------------------------------------------------------      
def _normalize_contractions(sentence_list):
    """
    it will correct each word in a sentence for slangs(ginger), emojis -> meaning, entity references and abbreviations(json file) : file can be manually modified above
    also makes everything lowercase (including EMPW,EMPU, EMPP, URL, etc)
    """
    #uses contraction_list (a json file) BE SURE TO IMPORT IT ALREADY
    norm_sents = []
    print("Normalizing contractions, abbreviations, slangs, emojis, character entities")
    for sentence in tqdm(sentence_list):
        norm_sents.append(_normalize_contractions_slang_emoji_entity(sentence))
    return norm_sents

def _normalize_contractions_slang_emoji_entity(text):
    """
    part1:normalizes english contractions.
    """
    contractions = contraction_list
    for word in text.split():
         if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    """
    part 2: using gingerit SMS slang correction:
    this is too slow and can take many hours for the whole dataset to run
    """
    sentence = text
    """
    part3: emoji and character entity reference conversion to meaning
    """
    emoticons = emot_obj.emoticons(sentence)
    for i in range(0,len(emoticons['value'])):
        sentence = sentence.replace(emoticons['value'][i], emoticons['mean'][i])
    """
    part4: make everything lowercase
    """
    sentence = sentence.lower()
    return sentence


In [12]:
contraction_list = json.loads(open("english_contractions.json.txt", 'r').read())
character_entity= {'&lt;3':'heart', '&amp;':'and','&quot;':' quote '}
contraction_list = {**contraction_list, **character_entity}

In [13]:
#assessment and examples:
# original_examples = ['hi @someone WATCH me #proud :) ;) ...... i h8 it bt w8 !!!!!  <3  wanna go &amp; &lt;3 tHeRe  &quot; bcs my finls clooooose &quot;bananas&quot; &amp; ']
original_examples=df.content[0:10]
preprocessed_examples = normalization_pipeline(original_examples)

##############################
Starting Normalization Process
Replacing -URL- , Replacing @MENTION and #HASHTAG, Reducing character repetitions, 
Simplifying punctuation, Removing whitespaces


100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 6662.91it/s]


Normalizing contractions, abbreviations, slangs, emojis, character entities


100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]

Normalization Process Finished
##############################


In [14]:
for example_index,example in enumerate(preprocessed_examples):
#     print(original_examples[example_index])
    print(original_examples.values[example_index])
    print(example)

    
#run preprocessing
df_original=df
df.content=normalization_pipeline(df.content.values ) #about 10 minutes to run

#save
df.to_csv('df_processed.csv',index=False)

Layin n bed with a headache  ughhhh...waitin on your call...
layin n bed with a headache ughh...waitin on your call...
Funeral ceremony...gloomy friday...
funeral ceremony...gloomy friday...
@dannycastillo We want to trade with someone who has Houston tickets, but no one will.
@men we want to trade with someone who has houston tickets, but no one will.
Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends
re-pinging @men: why did not you go to prom? bc my bf did not like my friends
I should be sleep, but im not! thinking about an old friend who I want. but he's married now. damn, &amp; he wants me 2! scandalous!
i should be sleep, but i am not! thinking about an old friend who i want. but he has married now. damn, and he wants me 2! scandalous!
Hmmm. http://www.djhero.com/ is down
hmm. -url- is down
@charviray Charlene my love. I miss you
@men charlene my love. i miss you
@kelcouch I'm sorry  at least it's Friday?
@men i am sorry at least it is friday?
ca

100%|█████████████████████████████████████████████████████████████████████████| 31313/31313 [00:01<00:00, 28351.72it/s]


Normalizing contractions, abbreviations, slangs, emojis, character entities


100%|████████████████████████████████████████████████████████████████████████| 31313/31313 [00:00<00:00, 237017.69it/s]


Normalization Process Finished
##############################


In [15]:
#load
df = pd.read_csv(r"english_contractions.json.txt")

In [16]:
df_processed = pd.read_csv(r"df_processed.csv")
print(df_processed)

         tweet_id  sentiment  \
0      1956967666    sadness   
1      1956967696    sadness   
2      1956968416    neutral   
3      1956968477      worry   
4      1956968487    sadness   
...           ...        ...   
31308  1753918954    neutral   
31309  1753919001       love   
31310  1753919005       love   
31311  1753919043  happiness   
31312  1753919049       love   

                                                 content  label data_type  
0      layin n bed with a headache ughh...waitin on y...      0     train  
1                    funeral ceremony...gloomy friday...      0     train  
2      @men we want to trade with someone who has hou...      1     train  
3      re-pinging @men: why did not you go to prom? b...      2     train  
4      i should be sleep, but i am not! thinking abou...      0     train  
...                                                  ...    ...       ...  
31308                                               @men      1     train  
31309  

In [17]:
spacy.load('en_core_web_sm')

In [18]:
dataset = Dataset.from_pandas(df_processed)
dataset = dataset.rename_column('content', "text")
dataset = dataset.rename_column('tweet_id', "id")

In [19]:
!pip install transformers
from transformers import AutoTokenizer

#Loads model and tokenizes stuff
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base") #Replace "" with relevant model

#Defines function for pre-processing - e.g. tokenising and truncating long phrases
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

#Maps pre-processing function to all of dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  9.55ba/s]


In [20]:
print(dataset[2])

{'id': 1956968416, 'sentiment': 'neutral', 'text': '@men we want to trade with someone who has houston tickets, but no one will.', 'label': 1, 'data_type': 'train'}


In [21]:
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=5) 
#Change model names and number of classes as is relevant.

training_args = TrainingArguments(
    #gradient_accumulation_steps=2,
    #gradient_checkpointing=True,
    #Memory-saving, more efficient way of reviewing dataset entries. Multiply by batch size for effective batch size.
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=32, #Decrease to save memory, but start at 16
    per_device_eval_batch_size=32, #Decrease to save memory, but start at 16
    optim="adafactor", #Optimiser - add to save memory
    num_train_epochs=6, #Start at five, but alter based on optimum loss
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset.filter(lambda x: x['data_type'] =='train'),
    eval_dataset=tokenized_dataset.filter(lambda x: x['data_type'] =='val'),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_accuracy, #Call accuracy score function
)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [23]:
print(tokenized_dataset[200])

{'id': 1957019458, 'sentiment': 'neutral', 'text': '#has 3wordsaftersex it never started...', 'label': 1, 'data_type': 'train', 'input_ids': [0, 85, 90, 698, 25844, 37734, 1871, 557, 18, 143, 4469, 13978, 28, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [27]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: data_type, sentiment, id, text. If data_type, sentiment, id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 30530
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 5730


Step,Training Loss
500,1.265900
1000,1.194100
1500,1.097700
2000,1.077800
2500,0.937100
3000,0.894000
3500,0.769600
4000,0.696300
4500,0.589700
5000,0.536400


Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-500\special_tokens_map.json
added tokens file saved in ./results\checkpoint-500\added_tokens.json
Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json
Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-1000\special_tokens_map.json
added tokens file saved in ./results\checkpoint-1000\added_tokens.json
Saving model checkpoint to ./results\checkpoint-1500
Configuration saved in ./results\checkpoint-1500\config.json
Model weights saved in ./results\checkpoint-1500\pytorch_model.bin
token

TrainOutput(global_step=5730, training_loss=0.848871063978052, metrics={'train_runtime': 1181.3175, 'train_samples_per_second': 155.064, 'train_steps_per_second': 4.851, 'total_flos': 4044608918136360.0, 'train_loss': 0.848871063978052, 'epoch': 6.0})

In [28]:
torch.save(model.state_dict(),'original_config.pt')

In [24]:
model.load_state_dict(torch.load('original_config.pt'))

<All keys matched successfully>

In [25]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, sentiment, id, data_type. If text, sentiment, id, data_type are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 783
  Batch size = 32


{'eval_loss': 0.32028037309646606,
 'eval_accuracy': 0.9003831417624522,
 'eval_runtime': 1.8566,
 'eval_samples_per_second': 421.74,
 'eval_steps_per_second': 13.466}

In [33]:
test = {'text': ['''

modern players are bad enough
at least i can focus on the game
commander you can't
because it's all social politics
no actual gameplay

''']}

test = preprocess_function(test)

test['input_ids'] = torch.tensor(test['input_ids'])
test['attention_mask'] = torch.tensor(test['attention_mask'])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
test['input_ids'] = test['input_ids'].to(device) 
test['attention_mask'] = test['attention_mask'].to(device)

print("predictions:", model(test['input_ids'], test['attention_mask']))

predictions: SequenceClassifierOutput(loss=None, logits=tensor([[ 2.6687, -0.3363,  3.4640, -1.7952, -3.3844]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [ ]:
#'sadness': 0, 'neutral': 1, 'worry': 2, 'love': 3, 'happiness': 4